In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from opym

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from opym

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs and calls the new validation/alignment function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    # --- NEW: Call the new processing function from opym ---
    # This function handles validation, alignment, and None assignment
    top_roi, bottom_roi = opym.process_rois_from_selector(
        mip_data,
        unaligned_rois,
        valid_threshold=1.0,
    )
    # --- END NEW ---

    if top_roi or bottom_roi:
        print("\nRun Cell 4 to visualize.")
    else:
        print("  Re-run Cell 2 to try again.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'mip_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from opym

from typing import cast

try:
    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs found. Please re-run Cell 2 and 3.")
    elif top_roi is None:
        print("ℹ️ Only Bottom ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know bottom_roi is not None here
        valid_bottom = cast(tuple[slice, slice], bottom_roi)
        opym.visualize_alignment(mip_data, valid_bottom, valid_bottom, vmin, vmax)
    elif bottom_roi is None:
        print("ℹ️ Only Top ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know top_roi is not None here
        valid_top = cast(tuple[slice, slice], top_roi)
        opym.visualize_alignment(mip_data, valid_top, valid_top, vmin, vmax)
    else:
        # Both are valid, show alignment
        opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)

    print("\nRun Cell 5 to save these ROIs to your log file.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # This is the central log file used by the CLI, as per the README.
    cli_log_file = Path("opm_roi_log.json")

    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs to save. Please re-run Cells 2 & 3.")
    else:
        opym.save_rois_to_log(
            log_file=cli_log_file,
            base_file=file_to_inspect,
            top_roi=top_roi,  # Can be None
            bottom_roi=bottom_roi,  # Can be None
        )
        print(f"✅ ROIs saved to {cli_log_file.name}")
        print("You can now use these ROIs for batch processing via the CLI,")
        print("or run Cell 6 to process this file immediately.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 6: Submit Cropping Job (Dynamic Channels & Remote Execution) ---

import importlib
import sys
from pathlib import Path
from typing import cast
import tifffile

import ipywidgets as widgets
from IPython.display import display

# --- 1. Import the opym package ---
try:
    import opym
    import opym.petakit  # Import the new petakit module
    from opym.utils import OutputFormat

    # Reload modules to pick up any changes
    importlib.reload(opym.utils)
    importlib.reload(opym.metadata)
    importlib.reload(opym.core)
    importlib.reload(opym.petakit) 
    importlib.reload(opym)
    print("Successfully re-loaded opym package.")

    # --- 2. Detect Channels Dynamically ---
    try:
        with tifffile.TiffFile(file_to_inspect) as tif:
            if hasattr(tif, "series") and len(tif.series) > 0:
                 shape = tif.series[0].shape
                 ndim = len(shape)
                 if ndim == 5:
                     n_channels_in = shape[2]
                 elif ndim == 4:
                     n_channels_in = shape[1]
                 else:
                     summary = tif.micromanager_metadata.get('Summary', {}) # type: ignore
                     n_channels_in = summary.get('Channels', 2)
            else:
                 n_channels_in = 2
    except Exception as e:
        print(f"Warning: Could not auto-detect channels ({e}). Defaulting to 4 inputs.")
        n_channels_in = 4

    n_excitations = n_channels_in // 2
    print(f"Detected {n_channels_in} Input Channels -> {n_excitations} Excitation(s).")

    # --- 3. Create Widgets ---
    output_format_widget = widgets.RadioButtons(
        options=[(f.value, f) for f in OutputFormat],
        description="Output Format:",
        value=OutputFormat.TIFF_SERIES,
        disabled=False,
    )

    rotate_widget = widgets.Checkbox(
        value=True, description="Rotate 90° (CCW)", disabled=False, indent=False
    )

    # --- Dynamic Checkbox Generation ---
    channel_checks = {}
    ui_rows = []
    
    ui_rows.append(widgets.Label(f"Channels to Output ({n_channels_in} inputs detected):"))

    for exc in range(n_excitations):
        base_id = exc * 4
        
        lbl = widgets.Label(f"--- Excitation {exc+1} (Camera 1 & Camera 2) ---")
        ui_rows.append(lbl)
        
        c_bot0 = widgets.Checkbox(value=True, description=f"C{base_id} (Bot, Cam 1)")
        c_top0 = widgets.Checkbox(value=True, description=f"C{base_id+1} (Top, Cam 1)")
        c_top1 = widgets.Checkbox(value=True, description=f"C{base_id+2} (Top, Cam 2)")
        c_bot1 = widgets.Checkbox(value=True, description=f"C{base_id+3} (Bot, Cam 2)")
        
        channel_checks[base_id]   = c_bot0
        channel_checks[base_id+1] = c_top0
        channel_checks[base_id+2] = c_top1
        channel_checks[base_id+3] = c_bot1
        
        ui_rows.append(widgets.HBox([c_bot0, c_top0]))
        ui_rows.append(widgets.HBox([c_top1, c_bot1]))

    channel_box = widgets.VBox(ui_rows)

    run_button = widgets.Button(
        description="Submit to PetaKit Queue", 
        button_style="primary", 
        icon="server"
    )
    
    # NEW: Status label for async updates
    status_label = widgets.Label(value="Ready to submit.")
    
    run_output = widgets.Output()

    display(
        widgets.VBox(
            [
                output_format_widget,
                rotate_widget,
                channel_box,
                widgets.HBox([run_button, status_label]), # Group button and label
                run_output,
            ]
        )
    )

    # --- 4. Define global vars to store paths ---
    processing_output_dir: Path | None = None
    processing_format: OutputFormat | None = None

    # --- 5. Define the function to run on button click ---
    def on_run_button_clicked(b):
        global processing_output_dir, processing_format

        with run_output:
            run_output.clear_output()

            try:
                # --- Get selected channels ---
                channels_to_output = []
                for cid, checkbox in channel_checks.items():
                    if checkbox.value:
                        channels_to_output.append(cid)

                channels_to_output.sort()
                
                if not channels_to_output:
                    print("❌ ERROR: No output channels selected.")
                    return

                print(f"Selected Output Channels: {channels_to_output}")

                # --- Validate ROIs ---
                need_top = any((ch % 4 in [1, 2]) for ch in channels_to_output)
                need_bottom = any((ch % 4 in [0, 3]) for ch in channels_to_output)

                if need_top and top_roi is None:
                    print("❌ ERROR: Selected Top channels, but no Top ROI drawn.")
                    return
                if need_bottom and bottom_roi is None:
                    print("❌ ERROR: Selected Bottom channels, but no Bottom ROI drawn.")
                    return

                # --- Run Job (REMOTE SUBMISSION) ---
                processing_format = cast(OutputFormat, output_format_widget.value)
                rotate_90_value = rotate_widget.value

                print("🚀 Submitting job to Petakit Queue...")
                
                job_path = opym.petakit.submit_remote_crop_job(
                    base_file=file_to_inspect,
                    top_roi=top_roi,
                    bottom_roi=bottom_roi,
                    channels=channels_to_output,
                    output_format=processing_format.value,
                    rotate=rotate_90_value
                )
                
                print(f"✅ Job Ticket Created: {job_path.name}")
                print("   The job is running in the background.")
                
                # Pre-calculate output path so subsequent cells have variables ready
                paths = opym.derive_paths(file_to_inspect, processing_format)
                processing_output_dir = paths.output_dir
                print(f"   Target Output Dir: {processing_output_dir.name}")
                
                # --- NEW: Async Background Monitor ---
                status_label.value = "⏳ Initializing..."
                opym.petakit.monitor_job_background(job_path, status_label)

            except NameError as e:
                print(f"❌ ERROR: Variable missing: {e}", file=sys.stderr)
            except Exception as e:
                print(f"❌ Unexpected error: {e}", file=sys.stderr)

    # --- 6. Link the button ---
    run_button.on_click(on_run_button_clicked)

except ImportError:
    print("❌ ERROR: Could not import opym package.", file=sys.stderr)

In [ ]:
# --- Cell 7: Run PyPetaKit5D Processing (Async Background) ---

import importlib
import re
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

import opym
import opym.metadata
import opym.petakit

# --- Reload modules ---
try:
    importlib.reload(opym.metadata)
    importlib.reload(opym.petakit)
    importlib.reload(opym)
except Exception:
    pass

# --- Create Widgets ---
petakit_button = widgets.Button(
    description="Submit PetaKit Job (Deskew/Rotate)", 
    button_style="success",
    icon="rocket"
)
# A label to show the background thread status
status_label = widgets.Label(value="Ready to submit.")
petakit_output = widgets.Output()

display(widgets.VBox([
    widgets.HBox([petakit_button, status_label]),
    petakit_output
]))

# --- Define the function to run on button click ---
def on_petakit_button_clicked(b):
    with petakit_output:
        petakit_output.clear_output()

        try:
            # 1. Validation
            if processing_output_dir is None or processing_format is None:
                print("❌ ERROR: 'processing_output_dir' not set. Please run Cell 6 first")
                return

            if processing_format != opym.OutputFormat.TIFF_SERIES:
                print("❌ ERROR: PetaKit step requires TIFF_SERIES output from Cell 6.")
                return

            # 2. Metadata Parsing (Z-Step)
            print("--- Parsing Metadata ---")
            log_file = next(processing_output_dir.glob("*_processing_log.json"), None)
            base_name = None
            
            if log_file:
                base_name = log_file.stem.replace("_processing_log", "")
            else:
                # Fallback: regex scan of files
                first_file = next(processing_output_dir.glob("*_C[0-9]_T[0-9][0-9][0-9].tif"), None)
                if first_file:
                    match = re.search(r"^(.*?)_C\d_T\d{3}\.tif$", first_file.name)
                    if match: base_name = match.group(1)

            z_step = 1.0
            if base_name:
                # Try finding metadata in parent folder
                metadata_file = processing_output_dir.parent / (base_name + "_metadata.txt")
                if metadata_file.exists():
                    z_step = opym.metadata.parse_z_step(metadata_file, default_z_step=1.0)
                    print(f"   Detected Z-step: {z_step} µm")
                else:
                    print(f"⚠️ Metadata not found ({metadata_file.name}). Defaulting to 1.0 µm.")
            else:
                print("⚠️ Could not determine base name. Defaulting to 1.0 µm.")

            # 3. Submit Async Job
            print(f"\n🚀 Submitting Job for: {processing_output_dir.name}")
            
            job_path = opym.petakit.submit_remote_deskew_job(
                input_dir=processing_output_dir,
                z_step_um=z_step,
                xy_pixel_size=0.136,
                sheet_angle_deg=31.8,
                deskew=True,
                rotate=True
            )
            
            print(f"✅ Ticket Created: {job_path.name}")
            print("   The job is now running in the background.")
            print("   You can continue working in other cells!")

            # 4. Start Background Monitor
            # This updates the widget label WITHOUT blocking the kernel
            status_label.value = "⏳ Initializing..."
            opym.petakit.monitor_job_background(job_path, status_label)

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not find required directories.")
            print(f"   Details: {e}")
        except Exception as e:
            print(f"\n❌ Unexpected error: {e}", file=sys.stderr)

petakit_button.on_click(on_petakit_button_clicked)

In [ ]:
# --- Cell 8: Load Final Processed Data ---
# This cell loads the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 7.

import ipywidgets as widgets
from IPython.display import display

import opym  # Make sure opym is imported

# --- Global variable to hold viewer parameters ---
viewer_params = None

# --- Create Widgets ---
load_dsr_button = widgets.Button(
    description="Load Final DSR Data", button_style="primary"
)
load_dsr_output = widgets.Output()


# --- Define the function to run on button click ---
def on_load_dsr_button_clicked(b):
    global viewer_params  # Make sure we're assigning to the global var

    with load_dsr_output:
        load_dsr_output.clear_output()

        try:
            # Check that the previous steps have been run
            if processing_output_dir is None:
                print(
                    "❌ ERROR: 'processing_output_dir' not set. Run Cell 6 & 7 first."
                )
                return

            # --- Automatically find the final DSR directory ---
            dsr_data_dir = processing_output_dir / "DSR"

            if not dsr_data_dir.exists():
                print(f"❌ ERROR: DSR directory not found at: {dsr_data_dir}")
                print("   Please run Cell 7 (PetaKit Processing) first.")
                return

            print(f"--- Loading data from: {dsr_data_dir.name} ---")

            # 1. Call the data loader
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(dsr_data_dir)

            # 2. Store parameters in the global variable
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)

            print(
                f"✅ Data loaded. Shape: T={t_min}-{t_max}, "
                f"Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}"
            )
            print(f"   Base name: {base_name}")
            print("\nYou can now run the viewer cells below.")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
load_dsr_button.on_click(on_load_dsr_button_clicked)
display(widgets.VBox([load_dsr_button, load_dsr_output]))

In [ ]:
# --- Cell 9: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 10: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")